In [ ]:
# Imports here
%matplotlib inline
%config InlineBackend.figure_format = 'retina'


import matplotlib.pyplot as plt
import time
import torch
from torch import nn
from torch import optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
from PIL import Image
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from tqdm import tqdm_notebook as tqdm
import time
import os, random
import copy

In [ ]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'



In [ ]:
# TODO: Define your transforms for the training, validation, and testing sets


data_transform = transforms.Compose([transforms.Resize(256),
                                transforms.CenterCrop(224),
                                transforms.ToTensor()])

dataset = datasets.ImageFolder(data_dir, transform=data_transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

In [ ]:
# TODO: Define your transforms for the training, validation, and testing sets


train_transforms = transforms.Compose([transforms.RandomRotation(15),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(250),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

valid_transforms= transforms.Compose([transforms.Resize(250),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

# TODO: Load the datasets with ImageFolder

train_data = datasets.ImageFolder(train_dir, transform=train_transforms)
test_data = datasets.ImageFolder(test_dir, transform=test_transforms)
valid_data = datasets.ImageFolder(valid_dir,transform=valid_transforms)
                            
image_datasets = train_data, test_data, valid_data                                     


# TODO: Using the image datasets and the trainforms, define the dataloaders


trainloader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=32)
validloader = torch.utils.data.DataLoader(valid_data, batch_size=32)


In [ ]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)
    
print(cat_to_name)
print("\n Length", len(cat_to_name))

In [ ]:
# TODO: Build and train your network

model = models.vgg16(pretrained=True)
model

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.vgg16(pretrained=True)

# turn off gradients for the model

for param in model.parameters():
    param.requires_grad = False
    
 # define our new classifier
from collections import OrderedDict
classifier = nn.Sequential(nn.Linear(25088, 1024),           
                           nn.ReLU(),
                           nn.Dropout(p=0.2),
                           nn.Linear(1024,512),
                           nn.ReLU(),
                           nn.Dropout(p=0.2),
                           nn.Linear(512 ,102),
                           nn.LogSoftmax(dim=1))

model.classifier = classifier

# define the criterion and optimizer and move the model to any device avaiable 

criterion = nn.NLLLoss()      # define the loss
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)    # optimizer

model.to(device);

In [ ]:
model

In [ ]:
# TODO: Do validation on the test set
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
epochs = 3
steps = 0
running_loss = 0
print_every = 42
model.to(device)
for epoch in range(epochs):
    for images, labels in trainloader:
        steps += 1
        # Move images and label tensors to the default device
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        logps = model.forward(images)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            valid_loss = 0
            model.eval()
            with torch.no_grad():
                for images, labels in validloader:
                    images, labels = images.to(device), labels.to(device)
                    logps = model.forward(images)
                    batch_loss = criterion(logps, labels)
                    test_loss += batch_loss.item()
                    valid_loss += batch_loss.item()
                    # Calculate accuracy
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                     
                   
                    
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Valid loss: {valid_loss/len(validloader):.3f}.. "
                  f"Valid accuracy: {accuracy/len(validloader):.3f}")
            running_loss = 0
            model.train()

In [ ]:
# TODO: Save the checkpoint 
print("Our model: \n\n", model, '\n')
print("The state dict keys: \n\n", model.state_dict().keys())

In [ ]:
model.class_to_idx = train_data.class_to_idx

In [ ]:
checkpoint = {'input_size': 25088,
              'output_size': 102,
              'hidden_layer1': 1024,
              'hidden_layer2':512,
              'learning rate':0.001,
              'classifier':classifier,
              'epochs':4,
              'optimizer':optimizer.state_dict,
              'class_to_idx':model.class_to_idx,
              'state_dict': model.state_dict()}

optimizer.state_dict
torch.save(checkpoint, 'checkpoint.pth')

In [ ]:
state_dict = torch.load('checkpoint.pth')
print(state_dict.keys())

In [ ]:
# TODO: Write a function that loads a checkpoint and rebuilds the model

def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    model = model.Network(checkpoint['input_size'],
                             checkpoint['output_size'],
                             checkpoint['hidden_layer1'],
                             checkpoint['hidden_layer2'],
                             checkpoint['learning rate'],
                             checkpoint['classifier'],
                             checkpoint['epochs'],
                             checkpoint['optimizer'])
    
    model.load_state_dict(checkpoint['state_dict'])
    
    return model

In [ ]:
model 

In [ ]:
from PIL import Image


def process_image(im):
    
   
    im = Image.open(im)
    width, height = im.size
    
    if width > height:
        ratio =float(width)/float(height)
        im.thumbnail((ratio*256,256))
        
    elif height>width:
        ratio =float(width)/float(height)
        im.thumbnail((256,ratio*256))
        
    new_width, new_height = im.size  #size of resized image
    
    left = (new_width-224)/2
    top = (new_height-224)/2
    right = (new_width +224)/2
    bottom = (new_height+224)/2
    im=im.crop((left, top, right, bottom))
    
    np_image=np.array(im)
    np_image=np_image/255
    
    means=np.array([0.485,0.456,0.406])
    std=np.array([0.229,0.224,0.225])
    
    np_image=(np_image-means)/std
    np_image=np_image.transpose((2,0,1))
    
    return np_image

In [ ]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = np.array(image).transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    if title:
        ax.set_title(title)
    
    return ax

In [ ]:
image='flowers/test/98/image_07753.jpg'
processed_image = process_image(image)
imshow(processed_image)
processed_image.shape

In [ ]:
''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    # TODO: Implement the code to predict the class from an image file

    
def predict(image_path, model, topk=5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    model.eval()
    
    image=process_image(image_path)
    image=torch.from_numpy(np.array([image])).float()
    
    image=Variable(image)
    image=image.to(device)
    output=model.forward(image)
     
    probability = torch.exp(output).data
    probs = torch.topk(probability,topk)[0].tolist()[0]
    indices = torch.topk(probability,topk)[1].tolist()[0]
    
    ind = []
    for i in range(len(model.class_to_idx.items())):
        ind.append(list(model.class_to_idx.items())[i][0])
        
    label = []
    for i in range(5):
        label.append(ind[indices[i]])

    return probs, label
       

In [ ]:
img = random.choice(os.listdir('flowers/test/98/'))
img_path= 'flowers/test/98/'+ img
with Image.open(img_path) as image:
        plt.imshow(image)
        
probs, classes = predict(img_path,model)
print(probs)
print(classes)
print([cat_to_name[x] for x in classes])

In [ ]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

In [ ]:
 TODO: Display an image along with the top 5 classes
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt 
import numpy as np 


probs, classes = predict(img_path, model)
max_index = np.argmax(probs)
max_probability = probs[max_index]
label=classes[max_index]

fig = plt.figure(figsize=(6,6))
ax1 = plt.subplot2grid((15,9),(0,0),colspan=9,rowspan=9)
ax2 = plt.subplot2grid((15,9),(9,2),colspan=5,rowspan=5)

image=Image.open(img_path)
ax1.axis('off')
ax1.set_title(cat_to_name[label])
ax1.imshow(image)

labels = []
for c1 in classes:
    labels.append(cat_to_name[c1])
    
y_pos=np.arange(5)
ax2.set_yticks(y_pos)
ax2.set_yticklabels(labels)
ax2.set_xlabel('probability')
ax2.invert_yaxis()
ax2.barh(y_pos, probs, xerr=0, align='center',color='blue')

plt.show()